In [60]:
from langchain.agents import Tool
import os

In [61]:
from api_keys import fetch_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = fetch_api_key("langsmith",requires_pass=False)
os.environ["MISTRAL_API_KEY"] = fetch_api_key("mistral",requires_pass=False)

Key loading successful.
Key loading successful.


In [62]:
from langchain_mistralai import ChatMistralAI

In [63]:
# llm = OllamaLLM(
#     model = 'llama2',
#     base_url = "http://localhost:11434"
# )
llm = ChatMistralAI(model = "mistral-large-2402")

In [64]:
import feedparser, requests

In [65]:
ARXIV_API_URL = "http://export.arxiv.org/api/query?"
class ArXivRetriever:
    def __init__(self, api_url="http://export.arxiv.org/api/query?"):
        self.api_url = api_url

    def fetch_papers(self, query, max_results=5):
        query_url = f"{self.api_url}search_query=all:{query}&max_results={max_results}"

        response = requests.get(query_url)
        feed = feedparser.parse(response.content)

        # Construct a list of dictionaries for JSON-like output
        papers = []
        for entry in feed.entries:
            temp = entry.id.split("/")
            pdf_url = "https://arxiv.org/pdf/" + temp[-2] + "/" + temp[-1] + ".pdf"
            paper = {
                "id": entry.id.split("/")[-1],
                "title": entry.title,
                "category": entry.category,
                "authors": [author.name for author in entry.authors],
                "author's comment": entry.get("arxiv_comment", "No comments available"),
                "published": entry.published,
                "summary": entry.summary,
                "link": entry.link,
                "pdf_url": pdf_url,
                'journal_ref' : entry.get("arxiv_journal_ref")
            }
            papers.append(paper)
        return papers

In [66]:
ret = ArXivRetriever()

In [67]:
search_tool = Tool(
    name= "Arxiv_Retriever",
    func= ret.fetch_papers,
    description = "Use this tool to search for research papers related to a topic."
)

In [68]:
# llm.invoke("Hello")

In [69]:
responses = search_tool.invoke("Attention is all yo need")

In [70]:
responses

[{'id': '1506.04932v1',
  'title': 'General Mixed Multi-Soliton Solutions to One-Dimensional Multicomponent\n  Yajima-Oikawa System',
  'category': 'nlin.SI',
  'authors': ['Junchao Chen', 'Yong Chen', 'Bao-Feng Feng', 'Ken-ichi Maruno'],
  "author's comment": '24 pages,7figs in Journal of the Physical Society of Japan (2015)',
  'published': '2015-06-16T11:59:22Z',
  'summary': 'In this paper, we derive a general mixed (bright-dark) multi-soliton solution\nto a one-dimensional multicomponent Yajima-Oikawa (YO) system, i.e., the\n(M+1)-component YO system comprised of M-component short waves (SWs) and\none-component long wave (LW) for all possible combinations of nonlinearity\ncoefficients including positive, negative and mixed types. With the help of the\nKP-hierarchy reduction method, we firstly construct two types of general mixed\nN-soliton solution (two-bright-one-dark soliton and one-bright-two-dark one for\nSW components) to the (3+1)-component YO system in detail. Then by exten

In [71]:
tools = [search_tool]

In [72]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [73]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)
config = {"configurable": {"thread_id":"abcd"}}

In [74]:
from langchain_core.messages import HumanMessage, SystemMessage

In [75]:
response = agent_executor.invoke({"messages" : [HumanMessage("Tell me about vision transformers, using any relevant research paper")]}, config)

In [76]:
response["messages"]

[HumanMessage(content='Tell me about vision transformers, using any relevant research paper', additional_kwargs={}, response_metadata={}, id='e4753bc6-b2b1-4e7e-bf0a-b7762e39dd42'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'hVebn5ZGU', 'type': 'function', 'function': {'name': 'Arxiv_Retriever', 'arguments': '{"__arg1": "vision transformers"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 97, 'total_tokens': 126, 'completion_tokens': 29}, 'model': 'mistral-large-2402', 'finish_reason': 'tool_calls'}, id='run-786a47e7-ebf0-47ea-8ee0-ffcadd7f651d-0', tool_calls=[{'name': 'Arxiv_Retriever', 'args': {'__arg1': 'vision transformers'}, 'id': 'hVebn5ZGU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 97, 'output_tokens': 29, 'total_tokens': 126}),
 ToolMessage(content='[{"id": "2204.07780v1", "title": "Towards Lightweight Transformer via Group-wise Transformation for\\n  Vision-and-Language Tasks", "category": "cs.CV", "authors": ["Gen Luo", "Yiyi Zho

In [77]:
agent_executor.invoke({"messages": HumanMessage("How does vision transformers are different than text Transformers?")}, config)

{'messages': [HumanMessage(content='Tell me about vision transformers, using any relevant research paper', additional_kwargs={}, response_metadata={}, id='e4753bc6-b2b1-4e7e-bf0a-b7762e39dd42'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'hVebn5ZGU', 'type': 'function', 'function': {'name': 'Arxiv_Retriever', 'arguments': '{"__arg1": "vision transformers"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 97, 'total_tokens': 126, 'completion_tokens': 29}, 'model': 'mistral-large-2402', 'finish_reason': 'tool_calls'}, id='run-786a47e7-ebf0-47ea-8ee0-ffcadd7f651d-0', tool_calls=[{'name': 'Arxiv_Retriever', 'args': {'__arg1': 'vision transformers'}, 'id': 'hVebn5ZGU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 97, 'output_tokens': 29, 'total_tokens': 126}),
  ToolMessage(content='[{"id": "2204.07780v1", "title": "Towards Lightweight Transformer via Group-wise Transformation for\\n  Vision-and-Language Tasks", "category": "cs.CV", "authors": ["Gen 